# WGS variant calling using GATK

- [ ] clarify the choice of reference genome
- [ ] clarify the choice of known sites
- [ ] modify the code to suit for multiple samples


## Tools


Tools necessary for WGS variant calling using GATK include:

Picard is a Java-based toolkit for manipulating high-throughput sequencing (HTS) data and formats such as SAM/BAM/CRAM and VCF.

GATK is a toolkit for variant discovery in high-throughput sequencing data. It is designed to work best with data generated by next-generation DNA sequencing technologies, but can be applied to other types of data as well. 

Samtools is a suite of programs for interacting with high-throughput sequencing data. It provides various utilities for post-processing alignments in the SAM, BAM and CRAM formats, such as indexing, variant calling, viewing alignments, and generating alignments in a per-position format. It also includes a large number of utilities for manipulating alignments in a variety of different ways, for example, to extract all reads that map to a particular region of the genome.

BWA is a software package for mapping low-divergent sequences against a large reference genome, such as the human genome. It consists of three algorithms: BWA-backtrack, BWA-SW and BWA-MEM. The first algorithm is designed for Illumina sequence reads up to 100bp, while the rest two for longer sequences ranged from 70bp to 1Mbp. BWA-MEM and BWA-SW share similar features such as long-read support and split alignment, but BWA-MEM, which is the latest, is generally recommended for high-quality queries as it is faster and more accurate. BWA-MEM also has better performance than BWA-backtrack for 70-100bp Illumina reads.  For all the algorithms, BWA first needs to construct the FM-index for the reference genome (the index command). Alignment algorithms are invoked with different sub-commands: aln/samse/sampe for BWA-backtrack, bwasw for BWA-SW and mem for the BWA-MEM algorithm.

FastQC is a quality control tool for high throughput sequence data. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis. It produces a simple HTML file with a summary of the results of the analysis, which can be viewed in a web browser.

Other tools that are useful for WGS variant calling include:
- Freebayes, a Bayesian genetic variant detector designed to find small polymorphisms, specifically SNPs (single-nucleotide polymorphisms), indels (insertions and deletions), MNPs (multi-nucleotide polymorphisms), and complex events (composite insertion and substitution events)
- bcftools, a set of utilities that manipulate variant calls in the Variant Call Format (VCF) and its binary counterpart BCF
- vcftools, a set of tools written in Perl and C++ for working with VCF files, such as those generated by the 1000 Genomes Project
- bedtools, a suite of utilities for genome arithmetic
- tabix, a generic indexer for TAB-delimited genome position files
- bgzip, a small utility that enables tabix to work with gzip-compressed files

## Reference genome

define the reference genome directory and files

In [ ]:
import os

REF_DIR = "/mnt/nas/wgs/hg38"
REF_URL = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz"
REF_FILE = os.path.join(REF_DIR, "hg38.fa")
KNOWN_SITES = os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf")
KNOWN_SITES_IDX = os.path.join(REF_DIR, "Homos_sapiens_assembly38.dbsnp138.vcf.idx")

download reference genome and index it

In [ ]:
# Download the reference genome if the file doesn't exist
if not os.path.exists(os.path.join(REF_DIR, "hg38.fa")):
    !wget -O $REF_DIR/hg38.fa.gz $REF_URL
    !gunzip $REF_DIR/hg38.fa.gz

# index the reference genome file before running haplotype caller
if not os.path.exists(os.path.join(REF_DIR, "hg38.fa.fai")):
    !samtools faidx $REF_DIR/hg38.fa

# Create dictionary file for the reference genome if it doesn't exist
if not os.path.exists(os.path.join(REF_DIR, "hg38.dict")):
    !gatk CreateSequenceDictionary R=$REF_DIR/hg38.fa O=$REF_DIR/hg38.dict

# index the reference genome file using BWA, if the index files don't exist
if not os.path.exists(os.path.join(REF_DIR, "hg38.fa.sa")):
    !bwa index $REF_FILE

SITE_URL = "https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf"
SITE_IDX_URL = "https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf.idx"

# Download known sites files for BQSR (Base Quality Score Recalibration) from GATK resource bundle if they don't exist
if not os.path.exists(os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf")):
    !wget -O $REF_DIR $SITE_URL

if not os.path.exists(os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf.idx")):
    !wget -O $REF_DIR $SITE_IDX_URL

# show the contents of the reference directory
!ls -halt $REF_DIR

## Reads

In [ ]:
READS_DIR = "/mnt/nas/wgs/RDD2206P0001"
READS_FILE1 = os.path.join(READS_DIR, "RDD2206P0001-WGS-1_R1.fastq.gz")
READS_FILE2 = os.path.join(READS_DIR, "RDD2206P0001-WGS-1_R2.fastq.gz")

## Quality control

This is a quick overview of the quality of the data. 
We will use FastQC to generate a report for each sample.

In [ ]:
# Run FastQC on the reads
!fastqc $READS_FILE1 $READS_FILE2 -o $READS_DIR

## Align reads to reference genome

the aligner `bwa aln` can trim the reads to remove low quality bases at the end of the reads. This is done using the `-q` option. The default value is 15. 

The BWA-MEM algorithm performs local alignment. It may produce multiple primary alignments for different part of a query sequence. This is a crucial feature for long sequences. However, some tools such as Picard’s `markDuplicates` does not work with split alignments. One may consider to use option `-M` to flag shorter split hits as secondary. This option is not recommended for Illumina short reads. 

In [ ]:
# Align the reads to the reference genome
!bwa mem -t 50 -M -R "@RG\tID:RDD2206P0001\tSM:RDD2206P0001" $REF_FILE $READS_FILE1 $READS_FILE2 > $READS_DIR/RDD2206P0001-WGS-1.sam

`-R` marks the complete read group header line. ’\t’ can be used in STR and will be converted to a TAB in the output SAM. The read group ID will be attached to every read in the output. The read group ID is used to identify the read group in the output file. The read group sample name is used to identify the sample in the output file.

### View SAM file

the SAM file is a text file that contains the alignment of the reads to the reference genome. SAM stands for Sequence Alignment/Map format.

In [ ]:
# Show the first 10 lines of the SAM file
!head -n 10 $READS_DIR/RDD2206P0001-WGS-1.sam

We can view the flag statistics of the SAM file using `samtools flagstat` command. The output of this command is a table that shows the number of reads that have a particular flag.

In [ ]:
# Show the flag statistics
!samtools flagstat $READS_DIR/RDD2206P0001-WGS-1.sam

The SAM file contains the following flags:

* 0x1: template having multiple segments in sequencing
* 0x2: each segment properly aligned according to the aligner
* 0x4: segment unmapped
* 0x8: next segment in the template unmapped
* 0x10: SEQ being reverse complemented
* 0x20: SEQ of the next segment in the template being reversed
* 0x40: the first segment in the template
* 0x80: the last segment in the template
* 0x100: secondary alignment
* 0x200: not passing quality controls
* 0x400: PCR or optical duplicate
* 0x800: supplementary alignment

The SAM file contains the following fields:

* QNAME: Query template NAME
* FLAG: bitwise FLAG
* RNAME: Reference sequence NAME
* POS: 1-based leftmost mapping POSition
* MAPQ: MAPping Quality
* CIGAR: CIGAR string
* RNEXT: Ref. name of the mate/next read
* PNEXT: Position of the mate/next read
* TLEN: observed Template LENgth
* SEQ: segment SEQuence
* QUAL: ASCII of Phred-scaled base QUALity+33

The SAM file needs to be sorted by the read name. This is done to make the file compatible with `MarkDuplicatesSpark` and `HaplotypeCallerSpark`.

## Convert SAM to BAM

the BAM file is a binary file that contains the alignment of the reads to the reference genome. BAM stands for Binary Alignment/Map format.  BAM is a compressed binary version of SAM.

In [ ]:
!samtools view -bS $READS_DIR/RDD2206P0001-WGS-1.sam > $READS_DIR/RDD2206P0001-WGS-1.bam

## Sort BAM

The BAM file is sorted by the read name. This is done to make the file compatible with `MarkDuplicatesSpark` and `HaplotypeCallerSpark`. 

In [ ]:
!samtools sort -@ 50 $READS_DIR/RDD2206P0001-WGS-1.bam -o $READS_DIR/RDD2206P0001-WGS-1.sorted.bam

In [ ]:
# Show the flag statistics
!samtools flagstat $READS_DIR/RDD2206P0001-WGS-1.sorted.bam

## Mark duplicates

Mark duplicates in the BAM file. This is done to remove PCR duplicates. PCR duplicates are reads that are generated from the same DNA fragment. PCR duplicates are generated during the PCR amplification step. PCR duplicates are removed to avoid overestimation of the coverage of the genome.

`MarkDuplicatesSpark` is a Spark version of `MarkDuplicates`. It is faster than `MarkDuplicates` and can be run on a cluster.

In [ ]:
!gatk MarkDuplicatesSpark -I $READS_DIR/RDD2206P0001-WGS-1.sorted.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam -M $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.metrics

In [ ]:
# Show the flag statistics
!samtools flagstat $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam

## Index BAM

Index the BAM file. This is done to make the file compatible with `HaplotypeCallerSpark`.

In [ ]:
!samtools index $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam

## Base quality score recalibration

Base quality score recalibration (BQSR) is a process that recalibrates the base quality scores of the reads. BQSR is done to correct for systematic errors in the base quality scores. BQSR is done using the `BaseRecalibrator` and `ApplyBQSR` tools. The `BaseRecalibrator` tool generates a recalibration table. The `ApplyBQSR` tool applies the recalibration table to the BAM file.

In [ ]:
!gatk BaseRecalibrator -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam -R $REF_FILE -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal_data.table --known-sites $KNOWN_SITES
!gatk ApplyBQSR -R $REF_FILE -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam --bqsr-recal-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal_data.table -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam

## Haplotype caller

The HaplotypeCaller tool is used to call variants. The HaplotypeCaller tool is a Spark version of the HaplotypeCaller tool. It is faster than the HaplotypeCaller tool and can be run on a cluster.

In [ ]:
!gatk HaplotypeCaller -R $REF_FILE -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.vcf

## Variant filtering

In [ ]:
!gatk VariantFiltration -R $REF_FILE -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.vcf --filter-expression "QD < 2.0" --filter-name "QD2" --filter-expression "FS > 60.0" --filter-name "FS60" --filter-expression "MQ < 40

## Variant annotation

In [ ]:
!gatk VariantAnnotator -R $REF_FILE -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.vcf --dbsnp $KNOWN_SITES

## Variant recalibration

In [ ]:
!gatk VariantRecalibrator -R $REF_FILE -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.vcf --tranches-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.tranches --rscript-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.plots.R --resource:hapmap,known=false,training=true,truth=true,prior=15.0 $HAPMAP --resource:omni,known=false,training=true,truth=false,prior=12.0 $OMNI --resource:1000G,known=false,training=true,truth=false,prior=10.0 $KG --resource:dbsnp,known=true,training=false,truth=false,prior=2.0 $KNOWN_SITES --an QD --an MQ --an MQRankSum --an ReadPosRankSum --an FS --an SOR --mode SNP --tranche 100.0 --tranche 99.9 --tranche 99.0 --tranche 90.0 --max-gaussians 4

## Apply variant recalibration

In [ ]:
!gatk ApplyVQSR -R $REF_FILE -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.vcf --ts-filter-level 99.0 --tranches-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.tranches --recal-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.filtered.annotated.recal.vcf --mode SNP

## The bash script

```{bash}
REF_DIR="/mnt/nas/wgs/hg38"
REF_URL="https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz"
REF_FILE="/mnt/nas/wgs/hg38/hg38.fa"
KNOWN_SITES_FILE="/mnt/nas/wgs/hg38/Homo_sapiens_assembly38.dbsnp138.vcf"
KNOWN_SITES_IDX_FILE="/mnt/nas/wgs/hg38/Homo_sapiens_assembly38.dbsnp138.vcf.idx"
READS_DIR="/mnt/nas/wgs/RDD2206P0001"
READS_FILE1="/mnt/nas/wgs/RDD2206P0001/RDD2206P0001-WGS-1_R1.fastq.gz"
READS_FILE2="/mnt/nas/wgs/RDD2206P0001/RDD2206P0001-WGS-1_R2.fastq.gz"

# Run FastQC on the reads
fastqc $READS_FILE1 $READS_FILE2 -o $READS_DIR

# Align the reads to the reference genome
bwa mem -t 50 -M -R "@RG\tID:RDD2206P0001\tSM:RDD2206P0001" $REF_FILE $READS_FILE1 $READS_FILE2 > $READS_DIR/RDD2206P0001-WGS-1.sam

# Convert the SAM file to BAM and sort it
samtools view -bS $READS_DIR/RDD2206P0001-WGS-1.sam > $READS_DIR/RDD2206P0001-WGS-1.bam
samtools sort -@ 50 $READS_DIR/RDD2206P0001-WGS-1.bam -o $READS_DIR/RDD2206P0001-WGS-1.sorted.bam

# Mark duplicates
gatk MarkDuplicatesSpark -I $READS_DIR/RDD2206P0001-WGS-1.sorted.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam --METRICS_FILE $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.metrics

# Show the flag statistics
samtools flagstat $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam > $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.flagstat

# Index the BAM file
samtools index $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam

# Create a BAI file
samtools index $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam

# Base quality score recalibration
gatk BaseRecalibratorSpark -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.grp -R $REF_FILE --known-sites $KNOWN_SITES_FILE

# Apply base quality score recalibration
gatk ApplyBQSRSpark -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam --bqsr-recal-file $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.grp -R $REF_FILE

# Collect alignment and insert size metrics
gatk CollectAlignmentSummaryMetrics -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.alignment_metrics -R $REF_FILE
gatk CollectInsertSizeMetrics -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.insert_size_metrics -H $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.insert_size_histogram.pdf -M 0.5

# Call variants
gatk HaplotypeCallerSpark -I $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.bam -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.vcf -R $REF_FILE --emit-ref-confidence GVCF --variant-index-type LINEAR --variant-index-parameter 128000

# extract the SNP variants
gatk SelectVariants -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.snp.vcf --select-type-to-include SNP

# extract the INDEL variants
gatk SelectVariants -V $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.vcf -O $READS_DIR/RDD2206P0001-WGS-1.sorted.dedup.recal.indel.vcf --select-type-to-include INDEL
```